# Data Transformation

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os
import pandas as pd

base_path = os.path.dirname(os.getcwd())
file_path = os.path.join(base_path,os.getenv("RAWDATA"))
quarter_path = os.path.join(base_path,os.getenv("Q_DATA"))

sheet_name = "月" 

df = pd.read_excel(file_path, sheet_name=sheet_name)
data_set={}
table_names=df.columns
for num in range(1,len(table_names)):
    avg_lsit=[]
    for data_num in range(0,len(df[table_names[num]]),3):
        data_avg=df[table_names[num]][data_num]+df[table_names[num]][data_num+1]+df[table_names[num]][data_num+2]/3
        avg_lsit.append(data_avg)
    data_set[table_names[num]]=avg_lsit

df_gdp = pd.DataFrame.from_dict(data_set)
df_gdp.to_csv(quarter_path, index=False, float_format='%.2f')  

# Data Consolidation

In [ ]:
final_path = os.path.join(base_path,os.getenv("TRAINDATA"))
sheet_name = "季" 

df = pd.read_excel(file_path, sheet_name=sheet_name)
df_t=pd.read_csv(quarter_path)

if len(df) == len(df_t):
    df_merged = pd.concat([df, df_t], axis=1)  
    print(df_merged.head())  
else:
    print("兩個 DataFrame 的行數不同，無法合併")

df_merged.to_csv(final_path, index=False, float_format='%.2f')  

       年份     GDP  平均匯率(元/美元)  國內投資毛額    投資率  工業生產指數  製造業銷售量指數  \
0  71年第1季  461774       37.91  102048  22.10   27.57     30.02   
1  71年第2季  488234       38.78  143589  29.41   30.35     33.54   
2  71年第3季  489504       39.65  117934  24.09   30.06     32.76   
3  71年第4季  498511       40.15  144192  28.92   29.86     32.70   
4  72年第1季  495361       39.97   99632  20.11   29.28     31.80   

   電力(企業)總用電量(十億度)  進口物價基本分類指數  出口物價基本分類指數     進口貿易總值     出口貿易總值  \
0             4.85      231.63      311.21  147089.00  135223.67   
1             5.46      231.87      315.17  172059.67  156827.00   
2             5.81      234.50      317.22  181502.67  148698.00   
3             5.66      234.53      315.38  168145.00  135928.00   
4             5.21      227.62      309.54  153110.00  131671.00   

   機械及電機設備進口值(十億元)  消費者物價指數   失業率  就業人數（千人）  總人口數(千人)  景氣領先指標綜合指數(點)  \
0              0.0   129.59  3.42  15826.00  42389.67          28.72   
1              0.0   130.53  4.12  15577.33  42570